In [1]:
import pytorch_lightning as pl
from omegaconf import OmegaConf, DictConfig
import nemo.collections.asr as nemo_asr
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
import datetime
from pathlib import Path
import torch
from sbb_project import consts
import copy
from pytorch_lightning.loggers import WandbLogger  
import wandb
from ruamel.yaml import YAML
import functools
import copy

[NeMo W 2023-01-02 16:53:56 optimizers:77] Could not import distributed_fused_adam optimizer from Apex


In [2]:
sweep_config = {
  "method": "random",   # Random search
  "metric": {           # We want to minimize `val_loss`
      "name": "val_wer",
      "goal": "minimize"
  },
  "parameters": {
        "lr": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -4.61    # exp(-4.61) = 1e-2
        },
        "epoch": {
            "distribution": "int_uniform",
            "min": 3,
            "max": 10
        },
        "freq_width": {
            "distribution": "uniform",
            "min": 25,  
            "max": 29     
        }
      
    }
}

In [3]:
train_manifest = [
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "none"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "neg10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "0"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "20"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "30"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "40"))),
    ]
val_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("val", "random"))
test_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("test", "random"))

In [4]:
def sweep_iteration():
    
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger(log_model='all')  # log final model
        
    trainer = pl.Trainer(gpus=1, max_epochs=wandb.config.epoch, logger=wandb_logger)
    
    # setup model - note how we refer to sweep parameters with wandb.config
    model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_de_citrinet_1024')
    
    model.set_trainer(trainer)
    
    model.cfg.train_ds.is_tarred = False
    
    model.cfg.train_ds.manifest_filepath = ','.join(train_manifest)
    model.cfg.validation_ds.manifest_filepath = str(val_manifest)
    model.cfg.test_ds.manifest_filepath = str(test_manifest)
    
    model.cfg.train_ds.batch_size = 8
    model.cfg.validation_ds.batch_size = 8
    model.cfg.test_ds.batch_size = 8
    
    model.cfg.optim.lr = wandb.config.lr
    model.cfg.spec_augment.freq_width = wandb.config.freq_width

    model.setup_training_data(model.cfg.train_ds)
    model.setup_validation_data(model.cfg.validation_ds)
    model.setup_test_data(model.cfg.test_ds)
    model.setup_optimization(model.cfg.optim)

    # train
    trainer.fit(model)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="ASR")
wandb.agent(sweep_id, function=sweep_iteration)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: v7qi12lc
Sweep URL: https://wandb.ai/unisg-ds-nlp/ASR/sweeps/v7qi12lc


wandb: Agent Starting Run: m9lgm488 with config:
wandb: 	epoch: 10
wandb: 	freq_width: 28.36039261652943
wandb: 	lr: 0.00017064384838052422
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marccgrau (unisg-ds-nlp). Use `wandb login --relogin` to force relogin


[NeMo W 2023-01-02 16:54:08 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:352: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
      rank_zero_warn(
    
[NeMo W 2023-01-02 16:54:08 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
      rank_zero_deprecation(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2023-01-02 16:54:09 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.12.0/stt_de_citrinet_1024/8743e4cecb8017ca2b49a035e07b2893/stt_de_citrinet_1024.nemo.
[NeMo I 2023-01-02 16:54:09 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.12.0/stt_de_citrinet_1024/8743e4cecb8017ca2b49a035e07b2893/stt_de_citrinet_1024.nemo
[NeMo I 2023-01-02 16:54:09 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2023-01-02 16:54:14 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


...


  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 140 M 
2 | decoder           | ConvASRDecoder                    | 1.1 M 
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WERBPE                            | 0     
------------------------------------------------------------------------
141 M     Trainable params
0         Non-trainable params
141 M     Total params
564.897   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:22:26 wer_bpe:187] 
    
[NeMo I 2023-01-02 23:22:26 wer_bpe:188] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:22:26 wer_bpe:189] predicted:prongierfahrt vom gleis maik zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:22:26 wer_bpe:187] 
    
[NeMo I 2023-01-02 23:22:26 wer_bpe:188] reference:spracheingabe
[NeMo I 2023-01-02 23:22:26 wer_bpe:189] predicted:spracheingabe


Training: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:22:58 wer_bpe:187] 
    
[NeMo I 2023-01-02 23:22:58 wer_bpe:188] reference:rangierfahrt von eins sieben nach gleis fünf antworten
[NeMo I 2023-01-02 23:22:58 wer_bpe:189] predicted:
[NeMo I 2023-01-02 23:23:26 wer_bpe:187] 
    
[NeMo I 2023-01-02 23:23:26 wer_bpe:188] reference:rangierfahrt von alpha zwei sieben sieben nach oscar null neun fünf
[NeMo I 2023-01-02 23:23:26 wer_bpe:189] predicted:fort zwei sie nach aus
